## Check Spark kernel connection

In [ ]:
sc

In [ ]:
SparkSession

In [ ]:
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")

### Check if it is able to connect to Hive

In [ ]:
cnbdata = spark.sql("SELECT * FROM schema.table LIMIT 50")


In [ ]:
pandas_cnbdata = cnbdata.toPandas()

# pandas_datehour_totals.set_index('datehour', inplace=True)
pandas_cnbdata.head()

In [ ]:
import warnings
warnings.filterwarnings('error', category=DeprecationWarning)

In [ ]:
!pip install --upgrade --force-reinstall flask-sqlalchemy sqlalchemy
!pip install pyspark-ai
!pip install google-cloud-aiplatform
!pip install langchain

In [ ]:
!pip install -U langchain-google-vertexai

In [ ]:
import sys,os,os.path
#Just to avoid the warnings
os.environ["OPENAI_API_KEY"] = "junk"

In [ ]:
import vertexai

PROJECT_ID = ""  # @param {type:"string"}
REGION = ""  # @param {type:"string"}

# Initialize Vertex AI SDK
vertexai.init(project=PROJECT_ID, location=REGION)

In [ ]:
# Utils
from langchain.schema import HumanMessage, SystemMessage
# from langchain.llms import VertexAI
from langchain_google_vertexai import VertexAI
from langchain.embeddings import VertexAIEmbeddings
# from langchain.chat_models import ChatVertexAI
from langchain_google_vertexai import ChatVertexAI
from google.cloud import aiplatform
import time
from typing import List


# LangChain
import langchain
from pydantic import BaseModel

from pyspark.sql.functions import expr


In [ ]:
print(f"LangChain version: {langchain.__version__}")

# Vertex AI

print(f"Vertex AI SDK version: {aiplatform.__version__}")

### Initialization 
#### (chat-bison@002 has little issues in udf usecase and Below model release date was 2023-07-10)

In [ ]:
chat1 = ChatVertexAI(
    model_name="chat-bison@001", max_output_tokens=1000, temperature=0.2
)

In [ ]:
chat2 = ChatVertexAI(
    model_name="chat-bison@002", max_output_tokens=1000, temperature=0.2
)

In [ ]:
from pyspark_ai import SparkAI

spark_ai = SparkAI(llm=chat1,verbose=True)
spark_ai.activate()  # active partial functions for Spark DataFrame

In [ ]:
cnbdata.ai.transform("get me all the names?").show()


In [ ]:
cnbdata.ai.transform("List all the married persons?").show()

### Only for udf we use chat-bison@001

In [ ]:
from pyspark_ai import SparkAI

spark_ai = SparkAI(llm=chat1,verbose=True)
spark_ai.activate() 

### Now lets create udf 

In [ ]:
@spark_ai.udf
def full_name(first_name: str, last_name: str) -> str:
    """combine first name and last name.
       If field is missing replace it by None. All imports should be inside function.
    """
    ...

In [ ]:
from pyspark.sql.functions import lit
from pyspark.sql.types import StringType
from pyspark.sql.functions import col, udf

spark.udf.register("full_name", full_name, returnType=StringType())
results = (
    cnbdata
    .withColumn("first_name", col("first_name"))
    .withColumn("last_name", col("last_name"))
    .withColumn("full_name", expr("full_name(first_name, last_name)"))
)

### test the udf

In [ ]:
results.head()

### Transform Accuracy Improvement: Vector Similarity Search

In [ ]:
@spark_ai.udf
def full_name(first_name: str, last_name: str) -> str:
    """combine first name and last name. Based on Gender add Prefix.
       If field is missing replace it by None. All imports should be inside function.
    """
    ...